# PLSE ETL - Part 2: Scraping PDFs to DB
Note: Using Sample Data Set (count=199)

In [168]:
from PyPDF2 import PdfFileReader

In [169]:
from os import listdir
from os.path import isfile, join

In [170]:
import re

In [171]:
path_to_dockets = '../tmp/sample_dockets'

In [172]:
dockets = [f for f in listdir(path_to_dockets) if isfile(join(path_to_dockets, f))]

In [173]:
keys = ['_c0', 'CountyName', 'DocketNumber', 'FiledDate', 'LastName', 'FirstName', 'MiddleName', 'City', 'State', 'ZipCode', 'OffenseTrackingNumber', 'GenderCode', 'RaceCode', 'BirthDate', 'OriginatingOffenseSequence', 'StatuteType', 'StatuteTitle', 'StatuteSection', 'StatuteSubSection', 'InchoateStatuteTitle', 'InchoateStatuteSection', 'InchoateStatuteSubSection', 'OffenseDisposition', 'OffenseDate', 'OffenseDispositionDate', 'OffenseDescription', 'CaseDisposition', 'CaseDispositionDate', 'OffenseGrade', 'DisposingJudge']

In [174]:
for k in keys:
    print(k)

_c0
CountyName
DocketNumber
FiledDate
LastName
FirstName
MiddleName
City
State
ZipCode
OffenseTrackingNumber
GenderCode
RaceCode
BirthDate
OriginatingOffenseSequence
StatuteType
StatuteTitle
StatuteSection
StatuteSubSection
InchoateStatuteTitle
InchoateStatuteSection
InchoateStatuteSubSection
OffenseDisposition
OffenseDate
OffenseDispositionDate
OffenseDescription
CaseDisposition
CaseDispositionDate
OffenseGrade
DisposingJudge


In [221]:
def compiled_pattern(pattern):
    return re.compile(r'{}'.format(pattern))

In [297]:
dn = re.compile(r'(?<=Docket\sNumber\:\s)\w{2}\-\d{2}\-\w{2}\-\d{7}\-\d{4}')

In [324]:
fd = re.compile(r'(?<=Date\sFiled\:\s{2})\d{2}\/\d{2}\/\d{4}')

In [367]:
keys = {
    'DocketNumber': (dn, None),
    'FiledDate': (fd, None)
}

In [368]:
def scrape_pdf(filename):
    path = '{}/{}'.format(path_to_dockets, filename)

    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        #info = pdf.getDocumentInfo()
        page_count = pdf.getNumPages()
        
        print(f)

        for page in range(page_count):
            page_data = pdf.getPage(page).extractText()
            
            for k in keys:
                
                if keys[k][1] is None:
                    print('Scraping PDF for:', k)
                
                    m = re.search(keys[k][0], page_data)

                    if m and m.group:
                        print('Value found:', m.group())
                        keys[k] = (keys[k][0], m.group())
                # else:
                    # the value has was previously successfully scrapped k `k`
            

In [369]:
f = scrape_pdf(dockets[0])

<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0307801-2003.pdf'>
Scraping PDF for: DocketNumber
Value found: MC-51-CR-0307801-2003
Scraping PDF for: FiledDate
Value found: 03/07/2003


In [370]:
print(keys)

{'DocketNumber': (re.compile('(?<=Docket\\sNumber\\:\\s)\\w{2}\\-\\d{2}\\-\\w{2}\\-\\d{7}\\-\\d{4}'), 'MC-51-CR-0307801-2003'), 'FiledDate': (re.compile('(?<=Date\\sFiled\\:\\s{2})\\d{2}\\/\\d{2}\\/\\d{4}'), '03/07/2003')}


In [371]:
for docket in dockets:
    # @todo @maybe validate `docket` type and path? or extend the declaration of `dockets` to validate?
    # @todo catch and handle PdfReadError: EOF marker not found, currently breaks loop
    try:
      scrape_pdf(docket)
    except:
      print("Something else went wrong") 

<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0307801-2003.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0108981-1996.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0224291-2002.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0100651-2003.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-1205031-1994.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-MD-9992548-2002.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-SU-0004193-1994.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-SU-0001804-1985.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-1003061-1978.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-1200421-1997.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0805391-1994.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0101801-1975.pdf'>
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0915521-1986.pdf'>
<_io.Buffere

AttributeError: 'NoneType' object has no attribute 'findall'